In [1]:
import pandas as pd
import requests
import pymongo

In [3]:
base_url = 'https://pokeapi.co/api/v2/'

In [11]:
# get all pokemon 
pokemon_url = base_url + 'pokemon/?offset=0&limit=964'
evolution_url = base_url + 'evolution-chain/?offset=0&limit=419'

In [12]:
pokemon_r = requests.get(pokemon_url)

In [16]:
pokemon_index = pokemon_r.json()['results']
# save raw data to my mongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.pokemon_db
# create colletions
col = db['pokemon_index']
col.insert_many(pokemon_index)

In [19]:
pokemon_details = []
for item in pokemon_index:
    url = item['url']
    pokemon_detail = requests.get(url).json()
    pokemon_details.append(pokemon_detail) 

In [21]:
pokemon_col = db['pokemon_data']
pokemon_col.insert_many(pokemon_details)

#Pokemon evolution

In [ ]:
evolution_chain = requests.get(evolution_url).json()['results']

def evolve_pokemon(url):
    url = url
    d = requests.get(url).json()['chain']
    chain = []
    chain.append(d['species']['name'])
    evolve = d['evolves_to']
    while(evolve != []):
        chain.append(evolve[0]['species']['name'])
        evolve = evolve[0]['evolves_to']
    return chain

evolution_data = []

for index, item in enumerate(evolution_chain):
    url = item['url']
    pokemon = evolve_pokemon(url)
    data = {'evl_id': index, 'chain': pokemon, 'stage': len(pokemon)}
    evolution_data.append(data)

# save raw data to my mongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')

# create database
db = client.pokemon_db

# create colletions
col = db['pokemon_evolve']
col.insert_many(evolution_data)